In [1]:
from collections import defaultdict, namedtuple
from importlib import reload
from pathlib import Path
from string import ascii_lowercase as LETTERS
from random import sample, choice, uniform
from time import time
from math import log, exp
import codecs
import csv
import logging
import random
import sys
import urllib.request
import zipfile

from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

from cryptogram_solver import data
from cryptogram_solver import solver
from cryptogram_solver import utils

In [2]:
project_dir = Path.cwd().parents[0]

In [20]:
Path.cwd()

PosixPath('/Users/chris/Documents/cryptogram-solver/notebooks')

In [4]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [5]:
docs = data.get_news_articles()

In [6]:
# news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=20180827)
# news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=20180827)

In [6]:
tokenizer_test = solver.Tokenizer()
tokenizer_test.tokenize("This is a sentence.")

{Token(ngrams=('<',), kind='char', n=1): 4,
 Token(ngrams=('<', 'a'), kind='char', n=2): 1,
 Token(ngrams=('<', 'a', '>'), kind='char', n=3): 1,
 Token(ngrams=('<', 'i'), kind='char', n=2): 1,
 Token(ngrams=('<', 'i', 's'), kind='char', n=3): 1,
 Token(ngrams=('<', 's'), kind='char', n=2): 1,
 Token(ngrams=('<', 's', 'e'), kind='char', n=3): 1,
 Token(ngrams=('<', 't'), kind='char', n=2): 1,
 Token(ngrams=('<', 't', 'h'), kind='char', n=3): 1,
 Token(ngrams=('>',), kind='char', n=1): 4,
 Token(ngrams=('a',), kind='char', n=1): 1,
 Token(ngrams=('a',), kind='word', n=1): 1,
 Token(ngrams=('a', '>'), kind='char', n=2): 1,
 Token(ngrams=('c',), kind='char', n=1): 1,
 Token(ngrams=('c', 'e'), kind='char', n=2): 1,
 Token(ngrams=('c', 'e', '>'), kind='char', n=3): 1,
 Token(ngrams=('e',), kind='char', n=1): 3,
 Token(ngrams=('e', '>'), kind='char', n=2): 1,
 Token(ngrams=('e', 'n'), kind='char', n=2): 2,
 Token(ngrams=('e', 'n', 'c'), kind='char', n=3): 1,
 Token(ngrams=('e', 'n', 't'), kin

In [7]:
reload(solver)
tokenizer = solver.Tokenizer(char_ngram_range=(1, 3), word_ngram_range=(1, 1), vocab_size=1000000)
tokenizer.fit(docs[:10000])
print(len(tokenizer.vocab))
print(tokenizer.totals)

100%|██████████| 10000/10000 [04:18<00:00, 38.66it/s]


109402
{('word', 1): 10030400, ('char', 1): 67208536, ('char', 2): 57178136, ('char', 3): 47147736}


In [8]:
tokenizer.totals

{('char', 1): 67208536,
 ('char', 2): 57178136,
 ('char', 3): 47147736,
 ('word', 1): 10030400}

In [9]:
# doc = cg.Doc(news_test['data'][0].lower())
doc = solver.Doc(docs[0].lower())
mapper = solver.Mapping()
mapper.scramble()
doc = mapper.translate(doc)
doc

'ecrdiunlbu  —   kbunpsrribuco pstwaoikcur dchs c use fscp edsu il kbxsr lb ldsip    dscold kcps ocerwil cnciurl lds bacxc cyxiuirlpclibu: ldsm xindl eiu. lds iukbxiun lpwxt cyxiuirlpclibu kbwoy kdbbrs lb ub obunsp ysfsuy lds sqskwlihs apcukd cnciurl lds rwil, edikd kdcoosunsr lds cyxiuirlpclibu’r cwldbpilm lb rtsuy aiooibur bf yboocpr bu dscold iurwpcuks rwariyisr fbp   cuy   cxspikcur, dcuyiun dbwrs pstwaoikcur c ain hiklbpm bu    irrwsr. awl c rwyysu obrr bf lds yirtwlsy rwariyisr kbwoy kbuksihcaom kcwrs lds dscold kcps tpbnpcx lb ixtobys, oschiun xiooibur bf tsbtos eildbwl ckksrr lb dscold iurwpcuks asfbps pstwaoikcur dchs tpstcpsy c pstocksxsul. ldcl kbwoy oscy lb kdcbr iu lds iurwpcuks xcpgsl cuy rtwp c tboilikco ackgocrd vwrl cr pstwaoikcur nciu fwoo kbulpbo bf lds nbhspuxsul. lb rlchs bff ldcl bwlkbxs, pstwaoikcur kbwoy fiuy ldsxrsohsr iu lds cegecpy tbrilibu bf cttpbtpicliun dwns rwxr lb lsxtbpcpiom tpbt wt lds bacxc dscold kcps oce, cunspiun kbursphclihs hblspr edb dchs assu 

In [14]:
[print(f'{x:0.3g}') for x in np.exp(np.linspace(3, -4, 5))];

20.1
3.49
0.607
0.105
0.0183


In [14]:
# '''
# Ideas:
# > Implement simulated annealing properly (single swap and decision), with a reasonable temp scheduler.
#   - This works! But need to play with temperature scheduler since results are sensitive to even small changes.
# > Implement simulated annealing but using softmax and subsetting swap options.
# > More swaps in the beginning, fewer later.
# '''

# def simulated_annealing(text, solver, tokenizer, num_epochs=10000, is_debug=True):
#     logger = logging.getLogger(__name__)
#     best_mapping = mapping = cg.Mapping()
#     doc = cg.Doc(text)
#     best_score = solver.score(doc)
#     epoch = 0
#     decisions = defaultdict(int)
#     temps = np.exp(np.linspace(0, -6, num_epochs))
#     decisions['good'], decisions['bad_keep'], decisions['bad_pass'] = 0, 0, 0
#     for temp in tqdm(temps):
#         improving = False
#         new_mapping = mapping.random_swap(doc.letters)
#         new_doc = new_mapping.translate(doc)
#         score = solver.score(new_doc)
#         score_change = score - best_score
#         if score_change < 0:
#             # logger.debug('updating (improvement)')
#             best_mapping = new_mapping
#             best_score = score
#             decisions['good'] += 1
#         elif exp(-score_change / temp) > uniform(0, 1):
#             # Break this out as different section just for debugging.
#             # logger.debug(f'updating (bad change): {score_change}')
#             best_mapping = new_mapping
#             best_score = score
#             decisions['bad_keep'] += 1
#         else:
#             decisions['bad_pass'] += 1
#             # logger.debug(f'keeping: {score_change}')
#         mapping = best_mapping
#         epoch += 1
#         if epoch % 1000 == 0:
#             logger.debug(f'{score:0.5g}, {mapping.mapping}, {mapping.translate(doc).text}')
#             logger.debug(sorted(list(decisions.items())))
#             logger.debug(pd.DataFrame(sorted(list(decisions.items()))))
#             decisions = defaultdict(int)
#     logger.info(f'\nfinal best ({epoch} epochs): {best_score:0.5g}')
#     return mapping.translate(doc).text

In [15]:
list(np.round(np.linspace(5, 1, 10)).astype(int))

[5, 5, 4, 4, 3, 3, 2, 2, 1, 1]

In [34]:
'''
Ideas:
> Implement simulated annealing properly (single swap and decision), with a reasonable temp scheduler.
  - This works! But need to play with temperature scheduler since results are sensitive to even small changes.
> Implement simulated annealing but using softmax and subsetting swap options.
> More swaps in the beginning, fewer later.
'''

def simulated_annealing(text, _, tokenizer, num_epochs=10000, is_debug=True):
    logger = logging.getLogger(__name__)
    best_mapping = mapping = solver.Mapping()
    doc = solver.Doc(text)
    best_score = solver.score(doc)
    epoch = 0
    decisions = defaultdict(int)
    temps = np.exp(np.linspace(0, -6, num_epochs))
    n_swap_list = np.round(np.linspace(3, 1, num_epochs)).astype(int)
    decisions['good'], decisions['bad_keep'], decisions['bad_pass'] = 0, 0, 0
    for temp, n_swaps in tqdm(zip(temps, n_swap_list)):
        improving = False
        new_mapping = mapping.random_swap(n_swaps)
        new_doc = new_mapping.translate(doc)
        score = solver.score(new_doc)
        score_change = score - best_score
        if score_change < 0:
            # logger.debug('updating (improvement)')
            best_mapping = new_mapping
            best_score = score
            decisions['good'] += 1
        elif exp(-score_change / temp) > uniform(0, 1):
            # Break this out as different section just for debugging.
            # logger.debug(f'updating (bad change): {score_change}')
            best_mapping = new_mapping
            best_score = score
            decisions['bad_keep'] += 1
        else:
            decisions['bad_pass'] += 1
            # logger.debug(f'keeping: {score_change}')
        mapping = best_mapping
        epoch += 1
        if epoch % 1000 == 0:
            logger.debug(f'{score:0.5g}, {mapping.key}, {mapping.translate(doc).text}')
            # logger.debug(sorted(list(decisions.items())))
            logger.debug(pd.DataFrame(sorted(list(decisions.items()))))
            decisions = defaultdict(int)
    logger.info(f'\nfinal best ({epoch} epochs): {best_score:0.5g}')
    return mapping.translate(doc).text

In [17]:
# doc = cg.Doc(doc.lower())

In [18]:
# [print(f'{x:0.3g}') for x in np.exp(np.linspace(-30, -5, 100))];

In [814]:
# # This is slower than the code above, and less consistent. But it does work.

# def choose_new_mapping(doc, mapping, score, proportion, temp):
    
#     swap_options = cg.get_swap_options(doc.letters, p=proportion)
#     mapping_options = [mapping.swap(*s) for s in swap_options]
#     scores = [solver.score(m.translate(doc)) for m in mapping_options]
    
#     # Add current without having to recompute score.
#     mapping_options.append(mapping)
#     scores.append(score)
    
#     probs = solver.softmax(-np.array(scores), temp)
#     # print(max(probs))
#     index = np.random.choice(range(len(probs)), size=1, p=probs)[0]
    
#     return mapping_options[index], scores[index]


# def simulated_annealing(text, solver, tokenizer, epochs, epochsbug=True):
#     logger = logging.getLogger(__name__)
#     best_mapping = mapping = cg.Mapping()
#     doc = cg.Doc(text)
#     epoch = 0
#     score = solver.score(mapping.translate(doc))
#     temps = np.exp(np.linspace(0, -6, epochs))
#     proportions = np.exp(np.linspace(-30, -5, epochs))
#     for temp, proportion in tqdm(zip(temps, proportions)):
#         mapping, score = choose_new_mapping(doc, mapping, score, proportion, temp)
#         epoch += 1
#         if epoch % 250 == 0:
#             logger.debug((
#                 f'{score:0.5g}, {mapping.mapping}, {mapping.translate(doc).text} '
#                 f'temp: {temp:0.5g}, proportion: {proportion:0.5g}'
#             ))
#     logger.info(f'\nfinal best ({epoch} epochs): {epoch:0.5g}')
#     return mapping.translate(doc).text

In [19]:
import spacy
nlp = spacy.load('en')

In [33]:
reload(solver)

log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.DEBUG, format=log_fmt)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.debug('hi')

sim_annealer = solver.Solver(tokenizer, logger=logger)

2019-02-08 22:33:44 DEBUG    hi


In [35]:
text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely love her when she smiles.'
sim_annealer.solve(text)

987it [00:01, 891.93it/s]2019-02-08 22:34:05 DEBUG    16.046, eizrosjlcmqhpukwvnbfdgtyxa, Tlbf bf wla fwedx et z vbdh. Wle idbau z dbqad zru udeprau wla pleha pedhu. Aru plbha fla heeoau fe fzu br mlewevdzmlf, I zsfehnwahx heqa lad plar fla fjbhaf.
2019-02-08 22:34:05 DEBUG    [('bad_keep', 357), ('bad_pass', 312), ('good', 331)]
1983it [00:02, 869.65it/s]2019-02-08 22:34:06 DEBUG    14.572, rnlahbipzxgtfyvcudokqwsjem, Tegw gw ley wlsan sm d kgac. Wes pagyr d agoya dbr rasvbyr ley vescy vsacr. Abr vegcy wey csstyr ws wdr gb heslskadhew, I dfwscqlycn csoy eya veyb wey wzgcyw.
2019-02-08 22:34:06 DEBUG    [('bad_keep', 263), ('bad_pass', 471), ('good', 266)]
2967it [00:03, 901.10it/s]2019-02-08 22:34:07 DEBUG    14.564, wvnueompscklthiqgrdjaxbfyz, Tnoi oi mne imfry fx u qorl. Wnf jroes u rober ucs srfaces mne anfle afrls. Acs anole ine lffkes if ius oc hnfmfqruhni, I uwifldmely lfbe ner anec ine igolei.
2019-02-08 22:34:07 DEBUG    [('bad_keep', 173), ('bad_pass', 642), ('good', 185)]
39

'This is the story of a girl. Who pried a river and drowned the whole world. And while she looked so sad in chotograchs, I absolutely love her when she smiles.'

In [26]:
# text = doc.text
# text = 'Rbo rpktigo vcrb bwucja wj kloj hcjd, km sktpqo, cq rbwr loklgo vcgg cjqcqr kj skhcja wgkja wjd rpycja rk ltr rbcjaq cj cr. -- Roppy Lpwrsborr'
text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely love her when she smiles.'
# text = 'Jail zesty vixen who grabbed pay from quack.'
# text = "I've found that when everyone rallies behind a cause, and when they learn their effort can contribute something bigger, they get engaged."
# text = str(random.choice(list(nlp(random.choice(articles)).sents)))
doc = solver.Doc(text.lower())
mapping = solver.Mapping()
mapping.scramble()
doc = mapping.translate(doc)
doc.text

'ifzn zn ifh nivtm vb c lztq. yfv jtzhk c tzxht cpk ktvyphk ifh yfvqh yvtqk. cpk yfzqh nfh qvvohk nv nck zp afvivltcafn, z cwnvqsihqm qvxh fht yfhp nfh ngzqhn.'

In [27]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG)
logger.debug('hi')

2019-02-08 20:53:57 DEBUG    hi


In [37]:
t0 = time()
# logging.basicConfig(level=logging.DEBUG)
text = simulated_annealing(doc.text, _, tokenizer, 7000)
t1 = time()
print(text)
print(t1 - t0)

AttributeError: module 'cryptogram_solver.solver' has no attribute 'score'